In [1]:
import numpy as np
import math
import os
import rioxarray as rxr
import time

from scipy import ndimage
from whitebox.whitebox_tools import WhiteboxTools

wbt = WhiteboxTools()
wbt.verbose = False


In [2]:
f1e = '08P_USGS_3DEP_3005_res1_00001.tif'
f1a = '08P_USGS_3DEP_3005_res1_00001_aspect.tif'
f1s = '08P_USGS_3DEP_3005_res1_00001_slope.tif'

In [3]:
def retrieve_raster(fpath):
    
    # fpath = '08P_USGS_3DEP_3005_res1_01710.tif'
    raster = rxr.open_rasterio(fpath, mask_and_scale=True)
    crs = raster.rio.crs
    affine = raster.rio.transform(recalc=False)
    return raster, crs, affine

In [4]:
Er, crs, affine = retrieve_raster(f1e)
E = Er.data[0]
print(E.shape)
E[38:48, 0:8].round(1)

(78, 81)


array([[   nan,    nan,    nan,    nan,    nan, 1292.4, 1297.2, 1305.2],
       [   nan,    nan,    nan,    nan, 1277.1, 1285.9, 1289.2, 1292.2],
       [   nan,    nan,    nan,    nan, 1277.1, 1271.3, 1272.5, 1271.4],
       [   nan,    nan,    nan, 1248.3, 1265.8, 1271.3, 1272.5, 1271.4],
       [   nan,    nan, 1232.6, 1235.7, 1241.2, 1249.6, 1255.2, 1267.5],
       [   nan, 1214.7, 1216.9, 1220.1, 1224.6, 1252.2, 1265.9, 1276. ],
       [1206.4, 1213.4, 1216.7, 1222. ,    nan,    nan,    nan,    nan],
       [   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan],
       [   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan],
       [   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan]],
      dtype=float32)

In [5]:
Sr, crs, affine = retrieve_raster(f1s)
Sr.data.shape

(1, 78, 81)

In [6]:
S = Sr.data[0]
S = -S + 90
S[38:48, 0:8].round(1)

array([[ nan,  nan,  nan,  nan,  nan, 74.5, 67.1, 64.7],
       [ nan,  nan,  nan,  nan, 76.4, 73. , 60.3, 54. ],
       [ nan,  nan,  nan,  nan, 74.3, 71.3, 70.2, 71.3],
       [ nan,  nan,  nan, 64.4, 54.3, 62. , 73.9, 77.1],
       [ nan,  nan, 69.3, 54.3, 51.5, 62.8, 68.5, 68.2],
       [ nan, 81.5, 72.9, 66. , 58.9, 66.6, 68.1, 68.7],
       [87.6, 83.2, 83.4, 86. ,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan]], dtype=float32)

In [ ]:
# S = -S + 90
S1_deg[38:48, 0:8].round(1)

In [50]:
Ar, crs, affine = retrieve_raster(f1a)
A = Ar.data[0]
# A = np.where(A < 90, 90-A, 360 - (A-90))
A[38:48, 3:10].round(1)

array([[  nan,   nan, 218.3, 215.9, 211.3, 209.6, 208.1],
       [  nan, 225.3, 196.8, 193.2, 188.6, 197.5, 196.2],
       [  nan, 210.7, 193.1, 174.3, 197.4, 232.3, 246.9],
       [220.8, 204. , 196.3, 188.7, 239.3, 278.8, 303.3],
       [215.6, 221.9, 215.5, 241.2, 271.7, 279. , 278.6],
       [212.6, 215.9, 262.1, 304.2, 307.1, 282.8, 294.5],
       [251.4,   nan,   nan,   nan,   nan,   nan,   nan],
       [  nan,   nan,   nan,   nan,   nan,   nan,   nan],
       [  nan,   nan,   nan,   nan,   nan,   nan,   nan],
       [  nan,   nan,   nan,   nan,   nan,   nan,   nan]], dtype=float32)

In [51]:
# convert to math convention angles
A = np.where(A < 90, 90-A, 360-(A-90))
A[38:48, 3:10].round(1)

array([[  nan,   nan, 231.7, 234.1, 238.7, 240.4, 241.9],
       [  nan, 224.7, 253.2, 256.8, 261.4, 252.5, 253.8],
       [  nan, 239.3, 256.9, 275.7, 252.6, 217.7, 203.1],
       [229.2, 246. , 253.7, 261.3, 210.7, 171.2, 146.7],
       [234.4, 228.1, 234.5, 208.8, 178.3, 171. , 171.4],
       [237.4, 234.1, 187.9, 145.8, 142.9, 167.2, 155.5],
       [198.6,   nan,   nan,   nan,   nan,   nan,   nan],
       [  nan,   nan,   nan,   nan,   nan,   nan,   nan],
       [  nan,   nan,   nan,   nan,   nan,   nan,   nan],
       [  nan,   nan,   nan,   nan,   nan,   nan,   nan]], dtype=float32)

In [52]:
A = np.where(A < 90, 90-A, 360 - (A-90))
x_mean = np.nanmean(np.cos(np.radians(A)))
y_mean = np.nanmean(np.sin(np.radians(A)))
mean_angle = np.degrees(np.arctan2(y_mean, x_mean))
# mean angle [-180, 180]
# if mean_angle < 0:
#     mean_angle = 90 - mean_angle
# else:
#     mean_angle = 270 + (180 - mean_angle)
# mean_A = (mean_angle + 360) % 360
mean_A = 90 - mean_angle
mean_A = (mean_A + 360) % 360
mean_A

194.77054595947266

In [53]:
mean_angle

-104.770546

In [ ]:
A1[38:48, 5:10].round(1)

In [ ]:
E[38:48, 3:10].round(1)

### dimensions, coordinates, etc.

This raster has shape (1, 4560, 3841).  

The first dimension (band) is the elevation data (z dimension)

The second and third values are the y and x dimensions, respectively.

The longitude (x direction) values are in the range 1.3M metres (for EPSG:3005), this corresponds to the x-dimension (east-west) which is 3841 wide.

The latitude (y direction) values are in the range 500K metres, this corresponds to the y-direction (north-south) which is 4560 high.


When we determine the slope and aspect, we can convolve a 3x3 matrix representing the cells surrounding each target cell:

|  |  |  |
|---|---|---|
| $Z_{-+}$ | $Z_{0+}$ | $Z_{++}$ |
| $Z_{-0}$ | $Z_{00}$ | $Z_{+0}$ |
| $Z_{--}$ | $Z_{0-}$ | $Z_{+-}$ |

Which is represented in matrix form as

|  |  |  |
|---|---|---|
| $Z_{0,0}$ | $Z_{0,1}$ | $Z_{0,2}$ |
| $Z_{1,0}$ | $Z_{1,1}$ | $Z_{1,2}$ |
| $Z_{2,0}$ | $Z_{2,1}$ | $Z_{2,2}$ |




We want to take the weighted average of three horizontal (dx) and vertical (dy) slopes, where the middle slope is given twice the weight of the outer slopes (sum to 1, so outer edges get 1/4 and middle gets 1/2).  We divide by $2dx$ because dx is the raster resolution, or the grid cell dimension in the x direction, and we are subtracting elevations at twice this distance.

$$dz_x = \frac{\frac{1}{4}\left( Z_{0,2} - Z_{0,0} \right) + \frac{1}{2}\left( Z_{1,2} - Z_{1,0} \right) + \frac{1}{4}\left( Z_{2,2} - Z_{2,0} \right)}{2 dx}$$



Which can be rewritten as:

$$dz_x = \frac{1}{8dx} \left[ \left(Z_{0,2} - Z_{0,0} \right) + 2\left( Z_{1,2} - Z_{1,0} \right) + \left( Z_{2,2} - Z_{2,0} \right) \right]$$

And similarly in the y direction:
$$dz_y = \frac{1}{8dy} \left[ \left(Z_{0,2} - Z_{2,2} \right) + 2\left( Z_{0,1} - Z_{2,1} \right) + \left( Z_{0,0} - Z_{2,0} \right) \right]$$

These can be written generally as computational expressions, for all matrix cells$i, j$:

`dz_x = ((Z[i-1,j+1] - Z[i-1,j-1]) + 2 * (Z[i,j+1] - Z[i,j-1]) + (Z[i+1,j-1] - Z[i+1,j+1])) / (8*dx)`

and

`dz_y = ((Z[i-1,j+1] - Z[i+1,j+1]) + 2 * (Z[i-1,j] - Z[i+1,j]) + (Z[i-1,j-1] - Z[i+1,j-1])) / (8*dy)`


See this post about performance of convolutions:

https://laurentperrinet.github.io/sciblog/posts/2017-09-20-the-fastest-2d-convolution-in-the-world.html

tl;dr, use numpy

In [ ]:
from numpy.fft import fft2, ifft2

In [ ]:
def calculate_slope(Z, dx=1, dy=1):
    """
    Z is a 3x3 matrix of elevation values, and we want to calculate the 
    weighted slope in both x and y directions, then convert it to slope in degrees.
    We then want to average all slope values for the raster
    """
    shape = Z.shape
    iy, ix = shape
    print(shape)
    S = np.empty_like(Z)
    for i in range(1, ix-1):
        for j in range(1, iy-1):
            # print(i, j, Z[i, j])
            dz_x = ((Z[i-1,j+1] - Z[i-1,j-1]) + 2 * (Z[i,j+1] - Z[i,j-1]) + (Z[i+1,j-1] - Z[i+1,j+1])) / (8*dx)
            dz_y = ((Z[i-1,j+1] - Z[i+1,j+1]) + 2 * (Z[i-1,j] - Z[i+1,j]) + (Z[i-1,j-1] - Z[i+1,j-1])) / (8*dy)
            max_slope_pct = np.sqrt(np.power(dz_x, 2) + np.power(dz_y, 2))
            max_slope_deg = (180 / np.pi) * np.arctan(max_slope_pct)
            # print(dz_x, dz_y, max_slope_pct, max_slope_deg)
            S[i,j] = max_slope_deg
    return np.mean(S)

Convolve a matrix by a 3x3 matrix "operator" to get the slope.

This is used in edge detection to determine the rate of change of color value, 
which is analogous to calculating slope of a surface.

In [ ]:
dy, dx = (abs(r) for r in Er.rio.resolution())
print(dy, dx)

In [ ]:
xf = np.array(
    [[1, 0, -1], 
      [2, 0, -2], 
      [1, 0, -1]], dtype=float) / (8.0 * dx)
yf = np.array([[-1, -2, -1], 
      [0, 0, 0], 
      [1, 2, 1]], dtype=float) / (8.0 * dy)

In [ ]:
from scipy.signal import convolve2d

In [ ]:
t0 = time.time()
ddx = convolve2d(E, xf, mode='same', boundary='symm')
ddy = convolve2d(E, yf, mode='same', boundary='symm')
t1 = time.time()
print(f'time to convolve: {t1-t0:.4f}')

In [ ]:
S1 = np.sqrt(np.add(np.power(ddx, 2.0), np.power(ddy, 2.0)))
S1_deg = (180/np.pi) * np.arctan(S1)
mean_S = np.nanmean(mean_S_deg)
mean_S

In [ ]:
A1 = (180 / np.pi)* np.arctan2(ddy, ddx)
# A1 = np.where(A1<0, 360+A1, A1)

A1[38:48, 3:10].round(1)

In [ ]:
A[38:48, 3:10].round(1)

In [ ]:
# calculate circular mean aspect
def calculate_circular_mean_aspect(aspect):
    """
    Calculate the circular mean of slope directions given 
    a matrix of slopes. Return circular mean aspect in degrees.
    """
    n_angles = np.count_nonzero(~np.isnan(aspect))
    sine_mean = np.divide(np.nansum(np.sin(np.radians(aspect))), n_angles)
    cosine_mean = np.divide(np.nansum(np.cos(np.radians(aspect))), n_angles)
    vector_mean = np.arctan2(sine_mean, cosine_mean)
    aspect_degrees = np.degrees(vector_mean)
    return aspect_degrees
    # if aspect_degrees + 180 > 360:
    #     return aspect_degrees - 180
    # else:
    #     return aspect_degrees + 180

In [ ]:

aspect
# mean_aspect_deg = calculate_circular_mean_aspect(aspect)/
# mean_aspect_deg
As = calculate_circular_mean_aspect(aspect)
As

In [ ]:
# This is the outlet of the basin, look at the polygon to verify
raster.data[0][75:85, 0:5].round(2)

In [ ]:
print(raster.data[0].shape, S.shape)

In [ ]:
# There's a big jump from 1515 to 1489, this is roughly a 
# 26m drop over 25m run (100% slope)
S[75:85, 0:5].round(2)

In [ ]:
# The slope corresponding to the big drop is mostly north to south
# but there's an east-west component as well
# the convention appears to indicate the uphill direction,
# counter-clockwise from due East
aspect[75:85, 0:5].round(1)

In [ ]:
# so now we just need to rotate by 180 degrees
# to give it the same sense as interpreted by "basin orientation"
# aspect = np.where(aspect + 180 > 360, aspect - 180, aspect + 180)
# aspect[75:85, 0:5].round(1)

In [ ]:
circ_mean = calculate_circular_mean_aspect(aspect)
circ_mean

In [ ]:
out_path = fpath.replace('.tif', '_slope.tif')
rp = '/home/danbot/Documents/code/22/basin_generator/references/'
wbt.aspect(
    fpath, 
    os.path.join(rp, out_path), 
    zfactor=None, 
    # callback=default_callback
)